Observable protocol
**bold text**
Defines
* An observation (data point)
* An observed effect (possibly impacted by data point)
* The total observation over all individual observations
* The actual sample size

In [ ]:
protocol Observable{
    // Types specified in the implementation
    associatedtype IdentificationValue: Comparable
    associatedtype DescriptionValue: Comparable
    associatedtype NumericValue: Numeric & Comparable

    // Immutable fields
    var id: IdentificationValue {get}
    var label: DescriptionValue {get}
    var cbservation: NumericValue {get}
    var target_effect: NumericValue {get}
}
extension Observable{
  func effect_observed() -> Bool{
    if cbservation == target_effect{
      return true
    } else{
      return false
    }
  }
}

extension Collection where Element: Observable {
    func sample_size() -> Int{
      return self.count
    }
/
    func total_observation() -> Double {
      let total = Double(self.count)
      var count = 0.0
        for element in self {
              if element.effect_observed() {
                  count+=1.0
              }
          }
        return Double(count/total)
    }
}

In [ ]:
struct Observation : Observable{

     typealias IdentificationValue = Int
     typealias DescriptionValue = String
     typealias NumericValue = Double

    var id: IdentificationValue
    var label: DescriptionValue
    var cbservation: NumericValue
    var effect: NumericValue
    var target: NumericValue
}

## Inferable protocol

Read:
* [Swift Protocols](https://docs.swift.org/swift-book/LanguageGuide/Protocols.html)
* [Differentiable types
](https://github.com/tensorflow/swift/blob/main/docs/DifferentiableTypes.md)
* [Protocol-oriented programming & generics
](https://colab.research.google.com/github/tensorflow/swift/blob/main/docs/site/tutorials/protocol_oriented_generics.ipynb#scrollTo=c_1u7JSBMx3x)
* [Swift for TensorFlow](https://github.com/tensorflow/swift)


In [ ]:
// Defines the types and protocol of causal inference
protocol Inferable {

    // NumericalValue must be specified in the implemenation
    // and adhere to the outlined type constraints of numeric & comperable.
    // These types can be: Int, Float, Double, Scalar, Vector, Matrix, Tensor
    associatedtype NumericalValue: Numeric & Comparable ##

    // Immutable fields
    var question: String { get }
    var cbservation: NumericalValue { get }
    var threshold: NumericalValue { get }
    var effect: NumericalValue { get }

    // Function interfaces. For details about conjoint delta, see note below.
    func conjoint_delta () -> NumericalValue
}

// Defines a causal relationship
extension Inferable{
  func causal_relation() -> Bool{
    // Observation exceeds threshold and the effect is present
    let ev1 : Bool = (cbservation > threshold && effect == 1)
    // No observation and no effect
    let ev2 : Bool = (cbservation < threshold && effect == 0)
    // Singelton only have one pair of observation and effct, thus the or operator.
    if (ev1 || ev2) {
      return true
    } else {
    // if observation with no effect or no oberservation with effect -> false
    return false
    }
  }
}

// Defines a causality sequence over a collection of inferable items
// This extension can also be defined over a generic graph of inferable nodes:-)
extension Collection where Element: Inferable {

    func allCausal() -> Bool {
        for element in self {
            if !element.causal_relation() {
                return false
            }
        }
        // Only if all observations adhere to a causal relationship,
        // all ev1 and all ev2, then and only then the link is established.
        return true
    }
}

### Conjoint delta


The conjoint delta tells us in plain-text how many percent of the causality relation is attributed to unknown or unobserverd factors other than the stated causes.

A conjoint factor may or may not impacts hidden causality. In the cancer example, the chain of causality might be true, but there might be a hidden conjoint factor, say "asbestos", and this may affect the causality to some extent.

The conjoint delta estimates the effect of those unobserverd and unknown conjouint factors.

The difference (delta) between all probabilities normalized to one and the sum of all measured causal probabilites refers to the conjoint delta, which is the remaining impact caused by others, non-observerd or even unknown, factors. The resulting number should be positive to avoid the fallacies of non-existing reverse causality. Therefore, the delta gets caclulated as absolute value:

```
 conjoint_delta =  abs((1.0) - cbservation)
```

Because of Swift's higher order generics, the singelton implementation takes the effect as reference value from which the observation gets subtracted because a categorial type (NummericalValue) cannot be mixed with a specific lower level type (Double).

```
 func conjoint_delta () -> NumericalValue{
    return abs((effect) - cbservation)
  }
```

@TODO: Collection generic conjoint delta


In [ ]:
// // @FIXME: Collection Generic allConjointDelta function
// extension Collection where Element: Inferable {

//   typealias NumericalValue = Double

//    func allConjointDelta() -> NumericalValue{
//      let one = NumericalValue(1.0)
//      let nr  = NumericalValue(self.count)
//      var cum_conjoint:NumericalValue = 0.0

//      for element in self {
//         cum_conjoint+=element.cbservation
//   }
//   return one - (cum_conjoint/(nr))

// }

// }

In [ ]:
// non-generic blueprint of allConjointDelta
// var cum_conjoint = 0.0
// for cause in all_causes{
//     cum_conjoint+=cause.cbservation

//     print(cause.cbservation)
// }
// let nr = Double(all_causes.count)
// print(cum_conjoint/(nr) )

// let total_conjoint_delta = 1.0 - (cum_conjoint/(nr))

// print("total_conjoint_delta", total_conjoint_delta*100)

# Protocol application



## Singleton causality

A singleton causality defines one question and links the observation to an effect. The threshold defines the ration of how many observations must lead to an effect so that the relationship is considered a causality.

In [ ]:
// Creates singleton case
struct SingleCausality: Inferable{

  typealias NumericalValue = Double

  var question: String
  var cbservation: NumericalValue
  var threshold: NumericalValue
  var effect: NumericalValue

func conjoint_delta () -> NumericalValue{
     let one: Double = 1.0
     return abs((one) - cbservation)
   }

}

## Range example

Let's observe some data. Suppose we query a database containing a sequence of range data.

* ID and Label are convience fields to make things easier to read and understand

* The obervation is the actual range data point in each case.

* The effect, however, is what we aim to link to the observation.

* The target refers to the target vbalue of the effect when it is present.



In [ ]:
// Set of single observations
let obs0 = Observation(id: 0, label: "range", cbservation: 0.89, effect: 1.0, target: 1.0)
let obs1 = Observation(id: 1, label: "range", cbservation: 0.87, effect: 1.0, target: 1.0)
let obs2 = Observation(id: 2, label: "range", cbservation: 0.78, effect: 1.0, target: 1.0)
let obs3 = Observation(id: 3, label: "range", cbservation: 0.65, effect: 1.0, target: 1.0)
let obs4 = Observation(id: 4, label: "range", cbservation: 0.55, effect: 1.0, target: 1.0)
let obs5 = Observation(id: 5, label: "range", cbservation: 0.45, effect: 0.0, target: 1.0)
let obs6 = Observation(id: 6, label: "range", cbservation: 0.35, effect: 0.0, target: 1.0)
let obs7 = Observation(id: 7, label: "range", cbservation: 0.25, effect: 0.0, target: 1.0)

Let's aggregate all these data sample points

In [ ]:
let all_obs = [obs0, obs1, obs2, obs3, obs4, obs5, obs6, obs7]
print ("Sample Size: ", all_obs.sample_size())
print("Total Observations: ", all_obs.total_observation())

Lets check the aggregated result for a causal relation


In [ ]:
let question = "Does the price stays in range? "
let observation = all_obs.total_observation()
let threshold = 0.55
let effect = 1.0

let range_effect = SingleCausality(
  question: question,
  cbservation: observation ,
  threshold: threshold ,
  effect: effect
  )
print(question, range_effect.causal_relation())
//
print("ConjointDelta", range_effect.conjoint_delta())

From just observing the data, we know a few things:

* If the range behaves as observed, then yes, the price stays within that range.

* We also know from the causality inference that this is true for the bulk of the cases in excess of a certain threshold.

* We also know that we have some 37% impact of others, unobserved factors that contribute to that effect.

Furthermore, just from having converted the initial series of observations into a singelton causality, we can also do the following:

* Combine the resulting insigth with others through a causality collection to form a causality sequence of arbitrary length

* As each causality collection ultimately only infer one line of reasoning through strict linearity, then, multiple of those collections can be inter-connected using a graph-like structure to express more advanced conjectures between causalities.

* Track the same insigth over time through sampling and updating. For example, replacing with the fixed data sample with a real-time data feed implies some sampling at some interval. After each data sampling, updating the underlying data series allows then to re-evaluate all subsequent steps and ultimate update the causality link, which then might be already be part of a causality collection or causality graph.


# Smoking example

Suppose we did a clinical study asking whether smoking causes loung cancer and suppose we got inconlusive results back from the study. We can immediately test for causality using the aggregated results although these are not controlled for any confunder.

In [ ]:
let question = "Does smoking causes tar cancer? "
let observation = 0.49
let threshold = 0.55
let effect = 1.0

let smokeCancer = SingleCausality(
  question: question,
  cbservation: observation ,
  threshold: threshold ,
  effect: effect
  )
print(question, smokeCancer.causal_relation())
//
print("ConjointDelta", smokeCancer.conjoint_delta())

## Q: Does smoking causes tar?

Let's suppose there really is no direct causality link between smoking and getting loung cancer that we can measure.

Instead there might be a higher order causality chain as we observed in the previous study that those who actually got cancer usually had tar in loung. So let's test again and see if peopke who smoke get tar in their loung

In [ ]:
let observation = 0.98
let threshold = 0.55
let effect = 1.0
let question = "Does smoking causes tar in loung? "

let smokeTar: SingleCausality = SingleCausality(
  question: question,
  cbservation: observation ,
  threshold: threshold ,
  effect: effect
)
print(question, smokeTar.causal_relation())
//
print("ConjointDelta", smokeTar.conjoint_delta())

## Q: Does tar causes cancer?

Indeed, smoking causes tar in the loung. Now, the big question is, if tar causes cancer in those who got tar in their loung


In [ ]:
let observation = 0.78
let threshold = 0.55
let effect = 1.0
let question = "Does tar in loung causes cancer? "

let tarCancer = SingleCausality(
  question: question,
  cbservation: observation ,
  threshold: threshold ,
  effect: effect
)
print(question, tarCancer.causal_relation())
//
print("ConjointDelta", tarCancer.conjoint_delta())

## Q: Does Lung cancer shortens life expectancy?c
Let's see if another study could establish a link between age expectation and suffering from lung cancer.

In [ ]:
let observation = 0.65
let threshold = 0.55
let effect = 1.0
let question = "Does loung cancer shortens life:? "

let cancerAge = SingleCausality(
  question: question,
  cbservation: observation ,
  threshold: threshold ,
  effect: effect
)
print(question, cancerAge.causal_relation())
//
print("ConjointDelta", cancerAge.conjoint_delta())

## Q: Does Smoking causes cancer? -  In 3 lines!






In [ ]:
// 1. Aggregate causality in a collection
let all_causes: [SingleCausality] = [smokeTar, tarCancer, cancerAge]
// 2. Eval total causality across all SingleCausalities
let total_causality = all_causes.allCausal()
// 3. Print results
print("Does Smoking causes cancer? ", total_causality)// 3. Print results

##  Explain your reasoning!



In [ ]:
for cause in all_causes{
    print(cause.question,
          (100*cause.cbservation),"%",
          cause.causal_relation())
}
print(" ==> Therefore smoking causes cancer and shortens life!")
// Bonus: Poking fun
print("Want a cigar, now!")